In [14]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Explore the dataset

In [15]:
from data_scientist import merge_dataset

In [16]:
cars_path, nocars_path = merge_dataset()
print('cars contain {} images'.format(len(cars_path)))
print('nocars contain {} images'.format(len(nocars_path)))

cars contain 8792 images
nocars contain 8968 images


In [17]:
config = {}
config['color_space'] = 'HLS'   # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
config['orient'] = 9              # HOG orientations
config['pix_per_cell'] = 8        # HOG pixels per cell
config['cell_per_block'] = 2      # HOG cells per block
config['hog_channel'] = 'ALL'     # Can be 0, 1, 2, or "ALL"
config['spatial_size'] = (16, 16) # Spatial binning dimensions
config['hist_bins'] = 32          # Number of histogram bins
config['spatial_feat'] = True     # Spatial features on or off
config['hist_feat'] = True        # Histogram features on or off
config['hog_feat'] = True         # HOG features on or off

In [18]:
color_space = config['color_space']
orient = config['orient']
pix_per_cell = config['pix_per_cell']
cell_per_block = config['cell_per_block']
hog_channel = config['hog_channel']
spatial_size = config['spatial_size']
hist_bins = config['hist_bins']
spatial_feat = config['spatial_feat']
hist_feat = config['hist_feat']
hog_feat = config['hog_feat']

In [19]:
from lesson_functions import extract_features

The 3 cells below only need to execute once

It's important to note here that when you are using a scaler to train a classifier, you want to only **fit** the scaler on the **training data**, and then **transform both the training and test sets using the scaler**. Why? If you provide both the training and test set to the scaler, you are allowing your model a peek into the values contained in the test set, and it's no longer as useful at generalizing to unseen data.

In [20]:
car_features = extract_features(cars_path, color_space=color_space, orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                                hog_channel=hog_channel, spatial_size=spatial_size, hist_bins=hist_bins, spatial_feat=spatial_feat, hist_feat=hist_feat,
                                hog_feat=hog_feat)
nocar_features = extract_features(nocars_path, color_space=color_space, orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                                hog_channel=hog_channel, spatial_size=spatial_size, hist_bins=hist_bins, spatial_feat=spatial_feat, hist_feat=hist_feat,
                                hog_feat=hog_feat)


features = np.vstack((car_features, nocar_features)).astype(np.float64)
labels = np.hstack((np.ones(len(cars_path)), np.zeros(len(nocars_path))))

d:\anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [21]:
Xscaler = StandardScaler()
Xscaler.fit(features)
scaled_features = Xscaler.transform(features)
X_train, X_valid, y_train, y_valid = train_test_split(scaled_features, labels, test_size=0.2, random_state=42)

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2, random_state=42)

# Fit from the training data
Xscaler = StandardScaler()
Xscaler.fit(X_train)

# Transform both training and test data
X_train_scaled = Xscaler.transform(X_train)
X_valid_scaled = Xscaler.transform(X_valid)

# Train the model
using SVM as classifier

In [23]:
svc = LinearSVC()
svc.fit(X_train_scaled, y_train)
print('The validation accuracy is {} %'.format((svc.score(X_valid_scaled, y_valid))))

The validation accuracy is 0.9901463963963963 %


In [24]:
from vehicle_classifier import save_trained_model
save_trained_model('./trained_models/SVM_HLS_1.p', svc, Xscaler, config)

Classifier saved to ./trained_models/SVM_HLS_1.p
